In [3]:
import PyPDF2
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Load trained model and vectorizer
model = joblib.load('pdf_type_classifier.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

def predict_pdf_type(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    features = vectorizer.transform([text])
    predicted_type = model.predict(features)[0]
    return predicted_type

# Example usage:
# pdf_path = 'sample.pdf'
# print("Predicted document type:", predict_pdf_type(pdf_path))

FileNotFoundError: [Errno 2] No such file or directory: 'pdf_type_classifier.pkl'

In [2]:
pip install PyPDF2 scikit-learn joblib

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
